In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_excel('./data/data.xlsx', skiprows=0)
del df['Unnamed: 0']

In [3]:
df = df.set_index("date", drop=True)

In [4]:
target = "electricity_price (PLN/MWh)"
target_mean = df[target].mean()
target_stdev = df[target].std()


In [5]:
df

,electricity_price (PLN/MWh),energy_demand (MW),energy_from_wind_sources (MW),is_holiday,code_of_the_day,electricity_price (PLN/MWh) lag24,electricity_price (PLN/MWh) lag48,electricity_price (PLN/MWh) lag72,electricity_price (PLN/MWh) lag96,electricity_price (PLN/MWh) lag120,...,energy_from_wind_sources (MW) lag168,energy_from_wind_sources (MW) lag336,energy_demand (MW) lag24,energy_demand (MW) lag48,energy_demand (MW) lag72,energy_demand (MW) lag96,energy_demand (MW) lag120,energy_demand (MW) lag144,energy_demand (MW) lag168,energy_demand (MW) lag336
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 01:00:00,117.00,14586,3938,1,0,98.00,121.03,121.22,125.60,113.52,...,5042,130,15048,15984,16093,16198,14063,13451,13482,16716
2018-01-01 02:00:00,113.59,14453,3876,1,0,82.00,121.03,120.00,120.42,113.00,...,5091,128,14295,15325,15383,15378,13525,12903,12995,16189
2018-01-01 03:00:00,97.00,13692,3897,1,0,76.14,121.03,119.60,116.30,105.00,...,4999,119,14110,14971,15229,15136,13204,12755,12587,16108
2018-01-01 04:00:00,89.00,13329,4091,1,0,74.70,121.03,119.60,116.30,111.39,...,4884,116,13961,14920,15160,15055,13243,12638,12276,16250
2018-01-01 05:00:00,75.00,13168,4197,1,0,73.78,121.03,121.22,119.20,117.54,...,4673,110,13910,15013,15372,15158,13319,12787,12228,16819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27 19:00:00,198.81,18254,5645,1,1,204.91,238.56,180.00,330.20,295.39,...,2653,197,17184,16568,16697,24065,24401,24650,20594,20903
2020-12-27 20:00:00,195.00,18242,5669,1,1,212.91,226.30,155.10,314.38,270.51,...,2611,235,17339,16608,16535,23724,24123,24361,20472,20810
2020-12-27 21:00:00,186.03,18021,5518,1,1,212.91,220.44,130.00,248.57,249.08,...,2435,250,17077,16381,16286,23029,23324,23697,20087,20090


In [6]:
for c in df.columns:
    mean = df[c].mean()
    stdev = df[c].std()

    df[c] = (df[c] - mean) / stdev
    

In [7]:
from prophet import Prophet

model = Prophet(weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')

In [8]:
ex_features = ['energy_demand (MW)',
               'energy_from_wind_sources (MW)', 
               'is_holiday',
               'code_of_the_day', 
               'electricity_price (PLN/MWh) lag24', 
               'electricity_price (PLN/MWh) lag48', 
               'electricity_price (PLN/MWh) lag72',
               'electricity_price (PLN/MWh) lag96',
               'electricity_price (PLN/MWh) lag120', 
               'electricity_price (PLN/MWh) lag144', 
               'electricity_price (PLN/MWh) lag168',
               'electricity_price (PLN/MWh) lag336', 
               'energy_demand (MW) lag24', 
               'energy_demand (MW) lag48',
               'energy_demand (MW) lag72', 
               'energy_demand (MW) lag96',
               'energy_demand (MW) lag120', 
               'energy_demand (MW) lag144',
               'energy_demand (MW) lag168', 
               'energy_demand (MW) lag336',
               'energy_from_wind_sources (MW) lag24',
               'energy_from_wind_sources (MW) lag48',
               'energy_from_wind_sources (MW) lag72',
               'energy_from_wind_sources (MW) lag96',
               'energy_from_wind_sources (MW) lag120',
               'energy_from_wind_sources (MW) lag144',
               'energy_from_wind_sources (MW) lag168',
               'energy_from_wind_sources (MW) lag336']


In [9]:
df = df.reset_index()

In [10]:
train_end = "2020-07-01 01:00:00"
test_start = "2020-10-01 01:00:00"

df_train = df.loc[df['date'] <= train_end]
df_test = df.loc[df['date'] >= test_start]

In [11]:
for feature in ex_features:
    model.add_regressor(feature)

    
#fit the model
model.fit(df_train[["date", "electricity_price (PLN/MWh)"] + ex_features].rename(columns={"date": "ds", "electricity_price (PLN/MWh)": "y"}))

forecast = model.predict(df_test[["date", "electricity_price (PLN/MWh)"] + ex_features].rename(columns={"date": "ds"}))

forecast_ = forecast.loc[:, forecast.columns.intersection(['ds','yhat'])]
forecast_ = forecast_.rename(columns={"ds": "date"})
df_test_ =  df_test.loc[:, df_test.columns.intersection(['date', "electricity_price (PLN/MWh)"])]
new_df = pd.merge(df_test_, forecast_, how='left', on=['date'])

In [13]:
new_df

,date,electricity_price (PLN/MWh),yhat
0,2020-10-01 01:00:00,-0.297834,-0.015512
1,2020-10-01 02:00:00,-0.297669,-0.074521
2,2020-10-01 03:00:00,-0.297834,-0.088247
3,2020-10-01 04:00:00,-0.297834,-0.032866
4,2020-10-01 05:00:00,-0.308882,0.001889
...,...,...,...
2106,2020-12-27 19:00:00,-0.366923,-0.791533
2107,2020-12-27 20:00:00,-0.429747,-0.832983
2108,2020-12-27 21:00:00,-0.577655,-0.939749
2109,2020-12-27 22:00:00,-0.763652,-1.116193


In [14]:
new_df = new_df.set_index('date')

#inverse transform the target and prediction columns
for c in new_df.columns:
    new_df[c] = new_df[c] * target_stdev + target_mean
    
#get performance results on the cv output
df_result = new_df

df_result = df_result.reset_index(drop=False)
df_result = df_result.rename(columns={'yhat': 'model_forecast'})

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

def calculate_metrics(df):
    return {'mae' : mean_absolute_error(df['electricity_price (PLN/MWh)'], df['model_forecast']),
            'rmse' : mean_squared_error(df['electricity_price (PLN/MWh)'], df['model_forecast']) ** 0.5,
            'r2' : r2_score(df['electricity_price (PLN/MWh)'], df['model_forecast']),
             'mape' : mean_absolute_percentage_error(df['electricity_price (PLN/MWh)'], df['model_forecast'])*100}


result_metrics = calculate_metrics(df_result)

In [16]:
result_metrics

{'mae': 16.863625240900504,
 'rmse': 21.700339835664657,
 'r2': 0.8613823600049286,
 'mape': 7.571258850283789}